> これは[キーボード #1 Advent Calendar 2022](https://adventar.org/calendars/7529)の20日目の記事の一部です。   
> トップページは[Pythonだけでキーボードを作る](https://5z6p.com/2022/12/21/ac2022/)です。

# ケースを作る

![目標のケース](../imgs/exported_case.png)

cadqueryを使って3Dモデリングし、3Dプリントできるケースを作ります。

google colaboratoryではOSの都合上cadqueryのビューアーが使えないため、binderでノートブックを開いています。   
binderはDockerを使ってOSや必要なライブラリを予め構成してjupyter notebookが使えます。   
そのため、ライブラリなどのインストールは必要ありません。

# cadqueryとは
[cadquery](https://github.com/CadQuery/cadquery)はpythonのコードを書いて3Dモデルを生成する環境です。
ビューアーには[jupyter_cadquery](https://github.com/bernhard-42/jupyter-cadquery)を使用します。

- cadquery https://github.com/CadQuery/cadquery
- cadqueryのドキュメント https://cadquery-ja.readthedocs.io/ja/latest/
- jupyter-cadquery https://github.com/bernhard-42/jupyter-cadquery

環境を用意します。下のセルを実行してバージョン情報が表示されるビューアーが開くことを確認してください。

In [ ]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay

enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

print()
versions()

cv = open_viewer("CadQuery", anchor="right")

## cadqueryを試してみる
簡単な直方体を生成してビューアーに表示します。

In [ ]:
# xy平面上から直方体を作ってエッジにフィレットをかける
box = cq.Workplane('XY').box(1, 2, 3).edges().fillet(0.1)
# デフォルトのビューアーに表示する
show(box)

# ケースを設計する
基板をネジで固定するトレイ型のケースを設計します。

## おおまかな外形
基板の形を元におおまかな外形を作ります。例に漏れず、定数を宣言して使いまわします。

In [ ]:
# 基板外形
PCB_WIDTH = 76.0
PCB_HEIGHT = 57.0
PCB_THICKNESS = 1.6

# ケースと基板のZ方向のマージン
CASE_MARGIN_TOP = 11.0
CASE_MARGIN_BOTTOM = 3.5

# ケースと基板のXY方向のマージン
CASE_MARGIN_PCB = 0.5

# ケースの厚み
CASE_FRAME = 2.0
CASE_BOTTOM = 3.0

INNER_HEIGHT = CASE_MARGIN_TOP + PCB_THICKNESS + CASE_MARGIN_BOTTOM
CASE_HEIGHT = INNER_HEIGHT + CASE_BOTTOM

# XY平面を基準に四角形を書いて押し出す
case = (
    cq.Workplane("XY")
    .rect(
        PCB_WIDTH + (CASE_FRAME + CASE_MARGIN_PCB) * 2,
        PCB_HEIGHT + (CASE_FRAME + CASE_MARGIN_PCB) * 2,
    )
    .extrude(CASE_BOTTOM + INNER_HEIGHT)
    .edges("|Z")
    .fillet(2)
    .edges("|X")
    .chamfer(1)
)

show(case)

## ケースの内側を切り取る
ケースの内側を切り取ってトレイの形にします。   
基準の平面を現在のcaseの面から選択してはじめます

In [ ]:
# caseの面からZ軸で一番上にあるものを基準とする
# 四角形を書いて内側の高さ分切り取る
case = (
    case.faces(">Z")
    .workplane()
    .rect(PCB_WIDTH + CASE_MARGIN_PCB * 2, PCB_HEIGHT + CASE_MARGIN_PCB * 2)
    .cutBlind(-INNER_HEIGHT)
)

show(case)

## ネジのボスと穴
ネジでとめるためのボスと穴を作ります。    
`tag("name")`で基準面を保存して使いまわしています。   
あらかじめネジ位置の座標の配列を用意して全ての箇所で同じ処理を繰り返します。

In [ ]:
# ネジ位置の座標
SCREW_POINTS = [(19, 9.5), (19, -9.5), (-19, -9.5), (0, 9.5)]

# caseの面からz軸で一番上にある面(ぎりぎり残ったケースの縁部分)を選択して
# INNER_HEIGHT分、下にオフセットした面を基準とする
# 各座標に半径2.5mmの円柱を作る
# 各座標に半径1.1mmの穴を開ける
# 基準面から1mm下にオフセットした面を新たな基準とし、四角い穴を底面まで開ける (ナットを入れる穴)
case = (
    case.faces(">Z")
    .workplane(offset=-INNER_HEIGHT)
    .tag("InnerBottom")
    .pushPoints(SCREW_POINTS)
    .circle(2.5)
    .extrude(CASE_MARGIN_BOTTOM)
    .workplaneFromTagged("InnerBottom")
    .pushPoints(SCREW_POINTS)
    .circle(1.1)
    .cutThruAll()
    .workplaneFromTagged("InnerBottom")
    .workplane(offset=-1)
    .pushPoints(SCREW_POINTS)
    .rect(4.2, 4.8)
    .cutBlind(-2)
)

show(case)

## USBコネクタ周り
USBコネクタの入る穴を開けます。   
USBコネクタは背面に開ける必要があるので基準にする平面はY軸上で一番遠い面を基準にします。   
前後で同じ距離の面がありますが、選択したところ背面になったのでそのままにしています。

In [ ]:
# USBコネクタの穴位置・寸法
USB_POS = (11.25, 1.2 + 3.15 / 2)
USB_HOLE_SIZE = [9, 3.2, 7.5]
USB_HOLE_MARGIN = 0.5
USB_CONN_SIZE = (11, 8)

# Y軸方向に一番遠い面を基準にする
# USBコネクタの穴は内側まで深めに切り取ってUSBコネクタが当たらないようにする
# USBケーブルのハウジングを避ける部分を切り取る
case = (
    case.faces(">Y")
    .workplane(centerOption="CenterOfMass")
    .center(
        PCB_WIDTH / 2 - USB_POS[0],
        CASE_HEIGHT / 2 - CASE_MARGIN_TOP - PCB_THICKNESS - USB_POS[1],
    )
    .tag("USBCutout")
    .rect(USB_HOLE_SIZE[0] + USB_HOLE_MARGIN, USB_HOLE_SIZE[1] + USB_HOLE_MARGIN)
    .cutBlind(-(CASE_FRAME + USB_HOLE_SIZE[2] + 2))
    .workplaneFromTagged("USBCutout")
    .rect(USB_CONN_SIZE[0], USB_CONN_SIZE[1])
    .cutBlind(-1)
)

show(case)

## OLEDの下のカバー
基板とOLEDの間にはさむカバーを設計します。   
別のオブジェクトとして作りますが、位置はあわせます。
カバーは両面テープで固定するので特にネジなどは用意しません。

In [ ]:
OLED_COVER_WIDTH = 19
OLED_COVER_HEIGHT = 19 * 2 - 3
OLED_COVER_THICKNESS = 10

# XY平面から基板の上にのる高さ分オフセットした面を基準にする
oled_cover = (
    cq.Workplane("XY")
    .workplane(offset=CASE_BOTTOM + CASE_MARGIN_BOTTOM + PCB_THICKNESS)
    .center(-PCB_WIDTH / 2, PCB_HEIGHT / 2)
    .tag("PCB_ORIGIN")
    .center(OLED_COVER_WIDTH / 2, -OLED_COVER_HEIGHT / 2)
    .rect(OLED_COVER_WIDTH, OLED_COVER_HEIGHT)
    .extrude(OLED_COVER_THICKNESS)
    .faces("Z")
    .tag("CASE_TOP")
    .edges(">Y or <X")
    .chamfer(1)
)

show(case, oled_cover)

## 3Dデータを出力する
最後に3Dプリントできるデータ形式で3Dモデルを出力します。

In [ ]:
cq.exporters.export(case, "case.stl")
cq.exporters.export(oled_cover, "oled_cover.stl")

# 3Dプリントする

生成したstlファイルを3Dプリントします。

![slicer](../imgs/slicer.png)

![基板とケース](../imgs/case.jpg)

![コネクタ](../imgs/case_usb.jpg)

印刷したあと基板をのせて確認しました。
コネクタ位置も完璧です。

これでケースは完成です！